In [1]:
import plotly.express as px
import pandas as pd

In [2]:
df = pd.read_csv('conferencemap.csv')

print(f"Map created with {len(df)} academic events from {df['Year'].min()} to {df['Year'].max()}")
print(f"Countries represented: {', '.join(sorted(df['Country'].unique()))}")

Map created with 19 academic events from 2017 to 2025
Countries represented:  UK, France, Germany, Ireland, Lithuania, Spain, UK


In [3]:
# Create a new column for location
df['Location'] = df['City'] + ', ' + df['Country']
# Create a new column for event count
df['Event Count'] = df.groupby('Location')['Event'].transform('count')

# Group events by location and concatenate event titles
df_grouped = df.groupby(['Location', 'Latitude', 'Longitude'])['Event'].apply(lambda x: '<br> '.join(x)).reset_index()
df_years = df.groupby(['Location', 'Latitude', 'Longitude'])['Year'].apply(lambda x: ', '.join(map(str, x))).reset_index()

# Merge the grouped data with the original data
df = df.merge(df_grouped, on=['Location', 'Latitude', 'Longitude'], suffixes=('', '_grouped'))
df = df.merge(df_years, on=['Location', 'Latitude', 'Longitude'], suffixes=('', '_years'))

In [4]:

# Create the map
fig = px.scatter_map(
    df,
    lat='Latitude',
    lon='Longitude',
    hover_name='Location',
    hover_data={
        'Location': True,
        'Event_grouped': True,
        'Year_years': True
    },
    size='Event Count',
    width=800,
    height=600,
    zoom=3.2,
    map_style= 'carto-positron'
    )

# Update marker styling
fig.update_traces(
    marker=dict(
        opacity=0.55,
        color="#7b61a7",
    ),
    hovertemplate='<b>%{customdata[0]}</b>'+
                  '<br>%{customdata[1]}'+
                  '<br>%{customdata[2]}'
)

# Update layout for better aesthetics
fig.update_layout(
    font=dict(size=12),
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=False  # Hide the legend
)

# Show the map
fig.show()
fig.write_html("conferencemap.html")
